In [22]:
import pandas as pd
import os
import warnings

pd.set_option('display.expand_frame_repr', False)
warnings.filterwarnings("ignore", message="Workbook contains no default style, apply openpyxl's default")

shop = 'NutriAsia'
sc_start_date = 'March 16'
sc_end_date = 'April 6'

# Order Data cannot be extracted from Lazada Seller Center in one go, so we had to break it down into multiple excel files
# All excel files from the same shop are stored in one folder
folder = f'Lazada Seller Center/{shop}'
excel_files = os.listdir(folder)

# Loop over all the excel files in the folder and import only the "orderNumber" and "status" columns
dfs = {}
for file in excel_files:
    file_path = os.path.join(folder, file)
    df = pd.read_excel(file_path, usecols=["orderNumber", "status", "createTime"])
    dfs[file] = df

# The result will be one DataFrame with "orderNumber" and "status" columns for each excel file
for file, df in dfs.items():
    print(file)
    print("Number of rows:", len(df))
    print(df.head(),'\n')

Lazada NutriAsia (Mar 17 - Mar 27).xlsx
Number of rows: 73215
          createTime      orderNumber     status
0  27 Mar 2024 23:59  816525702525203  delivered
1  27 Mar 2024 23:59  816525702525203  delivered
2  27 Mar 2024 23:59  816525702525203  delivered
3  27 Mar 2024 23:59  816505970453376  delivered
4  27 Mar 2024 23:59  816489186376290  confirmed 

Lazada NutriAsia (Mar 28 - Apr 6).xlsx
Number of rows: 68107
          createTime      orderNumber     status
0  06 Apr 2024 23:59  815600697348225  delivered
1  06 Apr 2024 23:59  815600697348225  delivered
2  06 Apr 2024 23:59  815600697348225  delivered
3  06 Apr 2024 23:59  815600697348225  delivered
4  06 Apr 2024 23:59  815600697348225  delivered 



In [51]:
# Merge into one DataFrame
seller_center = pd.concat(dfs.values(), keys=dfs.keys(), ignore_index=True)
print(f"Seller Center Lazada {shop} ({sc_start_date} - {sc_end_date})")
print("Number of rows:", len(seller_center))

Seller Center Lazada NutriAsia (March 16 - April 6)
Number of rows: 141322


In [52]:
# Remove duplicate entries
seller_center = seller_center.drop_duplicates(subset=['orderNumber'])
print(f"Seller Center Lazada {shop} ({sc_start_date} - {sc_end_date})")
print("Number of unique rows:", len(seller_center))

Seller Center Lazada NutriAsia (March 16 - April 6)
Number of unique rows: 58854


In [45]:
# Import Snowflake data into a DataFrame
snowflake = pd.read_csv(f'Snowflake/Snowflake Lazada {shop}.csv', usecols=['PLFM_ORD_ID','ORD_STAT_END', 'ORD_STAT_PLFM', 'ORDER_DATE'])

# Determine date range of the dataset
date_range = pd.date_range(start=snowflake.loc[:, 'ORDER_DATE'].min(), end=snowflake.loc[:, 'ORDER_DATE'].max())
sf_start_date = date_range[0].date()
sf_end_date = date_range[-1].date()

print(f"Snowflake Lazada {shop} ({sf_start_date} - {sf_end_date})")
print("Number of rows:", len(snowflake))

# Remove duplicates
snowflake.drop_duplicates(subset=['PLFM_ORD_ID'], inplace=True, ignore_index=True)
print("Number of unique rows:", len(snowflake), '\n')
print(snowflake.head())

Snowflake Lazada NutriAsia (2024-03-16 - 2024-04-06)
Number of rows: 149626
Number of unique rows: 62585 

       PLFM_ORD_ID ORD_STAT_END ORD_STAT_PLFM  ORDER_DATE
0  818002305158563    delivered     delivered  2024-03-31
1  816777909579847    delivered     confirmed  2024-03-28
2  811893627724522    delivered     delivered  2024-03-30
3  804429088792271    delivered     confirmed  2024-03-16
4  810659430337570    delivered     confirmed  2024-03-28


In [46]:
# Merge the Seller Center and Snowflake DataFrames so we could compare them
merged_df = pd.merge(seller_center, snowflake, left_on="orderNumber", right_on="PLFM_ORD_ID", how="inner")
print(f"Lazada {shop} Seller Center vs Snowflake ({sf_start_date} - {sf_end_date})")
print(merged_df.head(), '\n')
total = len(merged_df)
print("Number of rows:", total)

# Count the number of mismatched status
mismatched_status = merged_df[merged_df["status"] != merged_df["ORD_STAT_PLFM"]]
count_mismatched_status = len(mismatched_status)
print("Number of mismatched status:", count_mismatched_status)

# Calculate percent mismatched status
percent_mismatched_status = (count_mismatched_status / total) * 100
print("Percent mismatched status:", round(percent_mismatched_status, 2), "%")


Lazada NutriAsia Seller Center vs Snowflake (2024-03-16 - 2024-04-06)
          createTime      orderNumber     status      PLFM_ORD_ID ORD_STAT_END ORD_STAT_PLFM  ORDER_DATE
0  27 Mar 2024 23:59  816525702525203  delivered  816525702525203    delivered     delivered  2024-03-27
1  27 Mar 2024 23:59  816505970453376  delivered  816505970453376    delivered     delivered  2024-03-27
2  27 Mar 2024 23:59  816489186376290  confirmed  816489186376290    delivered     confirmed  2024-03-27
3  27 Mar 2024 23:59  810634851192589  delivered  810634851192589    delivered     delivered  2024-03-27
4  27 Mar 2024 23:59  816481196561736   canceled  816481196561736    cancelled      canceled  2024-03-27 

Number of rows: 58826
Number of mismatched status: 2184
Percent mismatched status: 3.71 %


In [53]:
# Check possible values of status column and count 
print(f"Seller Center Lazada {shop}")
print("Number of rows:", len(seller_center), '\n')
seller_center_status = seller_center['status'].value_counts()
print(seller_center_status, '\n')
print()
print(f"Snowflake Lazada {shop}")
print("Number of rows:", len(snowflake), '\n')
snowflake_ORD_STAT_PLFM = snowflake['ORD_STAT_PLFM'].value_counts()
print(snowflake_ORD_STAT_PLFM, '\n')
print()
print(f"Snowflake Lazada {shop}")
print("Number of rows:", len(snowflake), '\n')
snowflake_ORD_STAT_END = snowflake['ORD_STAT_END'].value_counts()
print(snowflake_ORD_STAT_END)

Seller Center Lazada NutriAsia
Number of rows: 58854 

status
confirmed                          33976
delivered                          17107
canceled                            6465
packed                               599
Package Returned                     280
shipped                              230
In Transit: Returning to seller       77
Package scrapped                      43
Damaged by 3PL                        35
ready_to_ship                         25
Lost by 3PL                           16
Buyer Delivery Failed                  1
Name: count, dtype: int64 


Snowflake Lazada NutriAsia
Number of rows: 785 

ORD_STAT_PLFM
packed           635
shipped           73
shipped_back      57
ready_to_ship     20
Name: count, dtype: int64 


Snowflake Lazada NutriAsia
Number of rows: 785 

ORD_STAT_END
pending      635
shipped       73
cancelled     57
rts           20
Name: count, dtype: int64


In [48]:
# Get non-end status rows (unpaid, packed, shipped, shipped back, ready tp ship)
non_end_status = ['unpaid', 'packed', 'shipped','shipped_back', 'ready_to_ship']

seller_center = seller_center[seller_center['status'].isin(non_end_status)]
snowflake = snowflake[snowflake['ORD_STAT_PLFM'].isin(non_end_status)]

print(f"Seller Center Lazada {shop} Non-End Status")
print("Number of rows:", len(seller_center), '\n')
seller_center_status = seller_center['status'].value_counts()
print(seller_center_status, '\n')
print()
print(f"Snowflake Lazada {shop} Non-End Status")
print("Number of rows:", len(snowflake), '\n')
snowflake_status = snowflake['ORD_STAT_PLFM'].value_counts()
print(snowflake_status)

Seller Center Lazada NutriAsia Non-End Status
Number of rows: 854 

status
packed           599
shipped          230
ready_to_ship     25
Name: count, dtype: int64 


Snowflake Lazada NutriAsia Non-End Status
Number of rows: 785 

ORD_STAT_PLFM
packed           635
shipped           73
shipped_back      57
ready_to_ship     20
Name: count, dtype: int64


In [49]:
# Merge the Seller Center and Snowflake DataFrames so we could compare them
final_df = pd.merge(seller_center, snowflake, left_on="orderNumber", right_on="PLFM_ORD_ID", how="inner")
print(f"Lazada {shop} Seller Center vs Snowflake ({sf_start_date} - {sf_end_date})")
print(final_df.head(), '\n')
print("Number of rows:", total)

# Count the number of mismatched status
mismatched_status = final_df[final_df["status"] != final_df["ORD_STAT_PLFM"]]
count_mismatched_status = len(mismatched_status)
print("Number of mismatched status:", count_mismatched_status)

# Calculate percent mismatched status
percent_mismatched_status = (count_mismatched_status / total) * 100
print("Percent mismatched status:", round(percent_mismatched_status, 2), "%")


Lazada NutriAsia Seller Center vs Snowflake (2024-03-16 - 2024-04-06)
          createTime      orderNumber  status      PLFM_ORD_ID ORD_STAT_END ORD_STAT_PLFM  ORDER_DATE
0  27 Mar 2024 23:54  816513929280408  packed  816513929280408      pending        packed  2024-03-27
1  27 Mar 2024 22:45  810576496717727  packed  810576496717727      pending        packed  2024-03-27
2  27 Mar 2024 21:53  816463913648420  packed  816463913648420      pending        packed  2024-03-27
3  27 Mar 2024 21:12  816433130028336  packed  816433130028336      pending        packed  2024-03-27
4  27 Mar 2024 21:08  816396391514252  packed  816396391514252      pending        packed  2024-03-27 

Number of rows: 58826
Number of mismatched status: 12
Percent mismatched status: 0.02 %
